<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab_11_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_11_3: longseq
## contents
- longseq introduce
  - 'hihello', charseq 와는 어떻게 다른지
- 긴 문장에서 sequence dataset 만들기(fixed size)
- adding FC layer and stacking RNN
- code run through
---
## longseq introduce
- 'hihello', charseq는 짧은 문장이라 문장 전체를 하나의 sample로 보고 진행
- 실제로 쓸모있는 RNN을 만드려면 긴 문장 dataset 필요
  - 아주 긴 문장을 하나의 input으로 사용할 수 없다.
  - longseq에서는 긴 문장을 특정 size로 잘라서 사용

## makeing sequence dataset from long sentence
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FNa2m9%2Fbtq0MT8SlQj%2FodFKBKQCGkAyVz80Qujsz0%2Fimg.png" width="500px" height="300px"></img>
- 특정 size의 윈도우를 오른쪽으로 한 칸씩 움직인다.
- 한 청크를 input_x로, 한 char만큼 shift한 청크를 y로
- 반복해서 dataset 만듦




In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
# data setting
x_data = []
y_data = []

# window를 이동하면서 data를 저장 
for i in range(0, len(sentence) - sequence_length):   # 가져올 수 있는 청크의 개수
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    # character dictionary에 있던 index 값으로 저장
    x_data.append([char_dic[c] for c in x_str])  # x string to index
    y_data.append([char_dic[c] for c in y_str])  # y string to index

# np.eye를 통해 one-hot vector화
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

## adding FC layer and stacking RNN

- 저번에는 단순히 RNN cell 하나만 있는 모델을 다룸
- 모델을 더 크게 만드는 방법 다양
  
<img src= "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbpWOVj%2Fbtq0Ojzd8AW%2FrkUjbyTIbN8brkVZJE9b30%2Fimg.png" width="250px" height="300px"></img>
- 이번 시간에는 간단하게 RNN을 stacking 하고 마지막에 FC layer 하나 추가
- RNN 두 번 통과한 다음 FC layer 쌓고 output_y


In [3]:
## declare RNN + FC
class Net(torch.nn.Module):     # Module을 상속받고 Net class로 정의, 두 가지 메소드(init, forward) 정의
   # constructor, 모듈이 내부적으로 어떤 하위 모듈을 쓰는지 정의(rnn, fc)
    def __init__(self, input_dim, hidden_dim, layers):    
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    # input_x 넣고 어떻게 계산할지 정의    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(dic_size, hidden_size, 2)

## code run through

In [ ]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

	# 모델 예측 결과 해석
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
       
        if j == 0: # 맨 처음에는 sequence length만큼의 문자 가져온다.
            predict_str += ''.join([char_set[t] for t in result])
       
        else: # result에서 마지막에 해당하는 문자열만 추가해준다. 맨 마지막빼고 기존 것과 겹침
            predict_str += char_set[result[-1]]

    print(predict_str)